In [ ]:
# from google.colab import drive # 挂载谷歌云盘
# drive.mount('/content/drive')
# !nvidia-smi # 显示显卡信息
# ''' 符号%代表一直生效，！代表执行完立马结束，不会生效，所以进入目录用% '''
# %cd /content/drive/MyDrive/timeSerise
# ''' 支持的 常用命令1.ls  2.wget  3.gdoint(int(int(int(w))))n  4.mkdir  5.pwd '''
# !ls
# !pip install patool
# !pip install sktime
# !pip install reformer_pytorch

In [2]:
from data_provider.data_creat import *
import akshare as ak
from datetime import datetime
import random
import numpy as np
import torch
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
# from exp.exp_imputation import Exp_Imputation
# from exp.exp_short_term_forecasting import Exp_Short_Term_Forecast
# from exp.exp_anomaly_detection import Exp_Anomaly_Detection
# from exp.exp_classification import Exp_Classification
# from data_provider.data_creat import *
# import akshare as ak
from torch.utils.tensorboard import SummaryWriter
from data_provider.data_loader import Dataset_Custom
from torch.utils.data import DataLoader

# 计算总体百分比变化的函数
def calculate_total_change(changes, start_day, end_day):
    # 总变化是通过顺序应用每天的变化计算的，
    # 从基础值100开始（代表股票的初始值）
    total_value = 100
    for change in changes[start_day-1:end_day]:
        total_value *= (1 + change / 100)
    # 然后根据最终值计算总体的百分比变化
    overall_change = ((total_value - 100) / 100) * 100
    return overall_change

In [3]:
class Args:
    '''基本配置'''
    # 选项：[long_term_forecast, short_term_forecast, imputation, classification, anomaly_detection]')
    task_name = 'long_term_forecast'
    model_id = 'stock_cy'
    des = '20240105' # 实验描述 20231210
    # 模型名称，选项：[Autoformer, Transformer, TimesNet]
    model = 'TimesNet'

    '''数据加载'''
    # 数据集类型,选项：[ETTh1,ETTh2,ETTm1,ETTm2,custom,m4,PSM,MSL,SMAP,SMD,SWAT,UEA]
    data = 'custom'
    test_path = 'raw_stock_cy_300798.csv'
    root_path = './dataset/Stock/'
    data_path = 'stock_cy_all_raw.csv'
    # 预测任务 M:多变量预测多变量, S:单变量预测单变量, MS:多变量预测单变量
    features = 'MS'
    # 目标列名，S或MS任务中的目标特征
    target = 'Change'
    # 时间采集粒度，选项：[s:秒, t:分钟, h:小时, d:天, b:工作日, w:周, m:月]
    freq = 'd'
    # 模型检查点的位置
    checkpoints = './checkpoints/'
    
    '''是否训练模型'''
    is_training = 0 # 设置为1则进行训练，设置为0则进行测试
    # 测试集的比例
    if is_training:
        test_ratio = [0.1,0.1] # test测试集、valid验证集的占比
        pred_once = False # 是否只预测一次，用于测试集数据
    else:
        test_ratio = [0.5,0.2]
        data_path = test_path
        pred_once = True # 是否只预测一次，用于测试集数据
    scale = False # dataset获取的源数据是否进行归一化
    back_training = False # 是否从上次训练的模型继续训练

    '''预测任务'''
    # 还原预测数据为原始数据
    inverse = scale
    # 输入序列长度,这是用于模型训练的输入序列的长度
    seq_len = 60
    # 开始标记长度,这是模型输出目标中有标签数据的长度，类似于滑动窗口的长度
    label_len = 20
    # 预测序列长度
    pred_len = 5
    # 季节模式（针对M4数据集）
    seasonal_patterns = 'Monthly'


    '''插补任务'''
    # 插补任务中数据丢失率
    mask_rate = 0.25

    '''异常检测任务'''
    # 异常检测中异常点占比
    anomaly_ratio = 0.25

    '''模型定义'''
    # 模型维度，线性层隐含神经元个数，数字越大显存用的越多
    d_model = 128
    # 前馈网络（FFN）的维度，数字越大显存用的越多
    d_ff = 128
    # TimesBlock 中傅里叶变换,频率排名前k个周期
    top_k = 5
    # Inception 中卷积核个数，用于 Inception
    num_kernels = 6
    # encoder 输入特征数
    enc_in = 16
    # decoder 输入特征数
    dec_in = enc_in
    # 输出通道数
    c_out = enc_in
    # 多头注意力机制
    n_heads = 8
    # encoder 层数，编码器层数
    e_layers = 2
    # decoder 层数，解码器层数
    d_layers = 1
    # 滑动窗口长度
    moving_avg = 20
    # 对 Q 进行采样，注意力因子数量
    factor = 3
    # 是否下采样操作 pooling，也就是对模型进行压缩
    distil = True
    # dropout 率
    dropout = 0.1
    # 时间特征嵌入方式,选项：[timeF, fixed, learned]
    embed = 'timeF'
    # 激活函数类型
    activation = 'gelu'
    # 是否输出 attention，也就是是否输出注意力权重
    output_attention = False

    '''优化'''
    # 并行核心数
    num_workers = 10
    # 实验轮数
    itr = 1
    # 训练迭代次数
    train_epochs = 500
    # batch size 大小
    batch_size = 256
    # early stopping 机制容忍次数
    patience = 3
    # 学习率
    learning_rate = 0.0001
    # 损失函数
    loss = 'MSE'
    # 学习率下降策略
    lradj = 'type1'
    # 使用混合精度训练
    use_amp = False

    '''GPU'''
    # 使用 gpu
    use_gpu = True
    gpu = 0
    # 使用多个 gpus
    use_multi_gpu = False
    # 多 gpu 的设备 id
    devices = '0,1,2,3'

    '''去平稳化投影仪参数'''
    # 投影仪的隐藏层维度（列表）
    p_hidden_dims = [128, 128]
    # 投影仪中的隐藏层数
    p_hidden_layers = 2
  
    '''股票数据获取'''
    fuquan = 'hfq'# 设置复权方式,adjust=空选择的不复权，qfq是前复权，应该用hfq后复权来进行量化分析
    period = 'daily' # 拉取时间周期{'daily', 'weekly', 'monthly'}
    start_date = '20151201'  # 20151201   20221021  下载数据的开始日期,1就是公司上市时间
    end_date = 1  # '20231220' 下载数据的结束日期,如果1则到最后一天,如果-1是昨天.
    label_n = 5 # 预测未来连续多少天的收益率
    pred_path = './dataset/Stock/pred/'
    
    # 预测目标y是否替换成1或者0
    zhangfu = 0.10  # 预测涨幅大于等于3%的为1，小于3%的为0
    label_ch = False  # 如果是True ，预测n天以后上涨大于变量zhangfu为1，小于为0
    
    # 数据修剪
    start = 103 # 删除前24行（start=25），因为macd算不出来
    end = 0 # (end = label_n if all else 0)删除最后部分需要预测天数label_n的数据，算出来是0.如果单只股票预测，那么不需要删除
    # final_data_feat =  ['index', 'Volume','Tom_Chg'] # 删除不需要列的标签
    final_data_feat =  ['index', 'Volume','Tom_Chg','Open','Low','High'] # 删除不需要列的标签

# 创建参数对象
args = Args()

# 设置随机种子以确保结果可重现
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

# 创建股票数据列表
stock_down = ak.stock_cy_a_spot_em() # 创业板实时数据
stock_list = stock_down[~stock_down['名称'].str.contains("退|ST") & (stock_down['流通市值'] <= 1e11) & (stock_down['总市值'] >= 45e8)] # 去除退市和ST股票
file_name_cy = 'Stock_list_cy.csv'# 保存数据，编码格式为utf-8
stock_list.to_csv(args.pred_path + file_name_cy,index=False,encoding='utf-8-sig')

# 读取股票列表
stock_list = pd.read_csv(args.pred_path + file_name_cy) # 读取股票列表
# 将股票代码的数字转换为字符串列表
stock_list = [str(code) for code in stock_list['代码'].tolist()]

# stock_list = ['000158','300798'] # 自定义股票列表

### 初始化模型参数

In [4]:
# 检查并设置 GPU
args.use_gpu = torch.cuda.is_available() and args.use_gpu
if args.use_gpu:
    print("使用 GPU.")
    total_cuda_devices = torch.cuda.device_count()  # 获取系统中可用的 GPU 总数
    print(f"系统中总共有 {total_cuda_devices} 个 CUDA 设备可用。")
    if args.use_multi_gpu:
        args.devices = args.devices.replace(' ', '')
        device_ids = args.devices.split(',')
        args.device_ids = [int(id_) for id_ in device_ids]
        args.gpu = args.device_ids[0]

        # 打印多 GPU 使用情况
        print(f"使用多个GPU: {args.device_ids}")
        device = torch.device(f"cuda:{args.gpu}" if args.use_gpu else "cpu")
        print(f"Primary GPU (cuda:{args.gpu}) is in use.")
    else:
        args.gpu = 0
        device = torch.device("cuda" if args.use_gpu else "cpu")
        print("使用单个 GPU.")
else:
    device = torch.device("cpu")
    print("使用 CPU.")


# 选择合适的实验类
if args.task_name == 'long_term_forecast':
    Exp = Exp_Long_Term_Forecast
# elif args.task_name == 'short_term_forecast':
#     Exp = Exp_Short_Term_Forecast
# elif args.task_name == 'imputation':
#     Exp = Exp_Imputation
# elif args.task_name == 'anomaly_detection':
#     Exp = Exp_Anomaly_Detection
# elif args.task_name == 'classification':
#     Exp = Exp_Classification
else:
    Exp = Exp_Long_Term_Forecast  # 默认情况
    
    
ii = 0
setting = '{}_{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.task_name, # 任务名称
            args.model_id, # 模型id
            args.model, # 模型名称
            args.data, # 数据集名称
            args.features, # 预测任务
            args.seq_len, # 输入序列长度
            args.label_len, # 开始标记长度
            args.pred_len, # 预测序列长度
            args.d_model, # encoder 输入特征数
            args.n_heads, # 多头注意力机制
            args.e_layers, # encoder 层数
            args.d_layers, # decoder 层数
            args.d_ff, # FFN 层隐含神经元个数
            args.factor, # 对 Q 采样的因子数
            args.embed, # 时间特征嵌入方式
            args.distil, # 是否下采样操作 pooling
            args.des,  # 实验描述
            ii) # 实验轮数

使用 GPU.
系统中总共有 1 个 CUDA 设备可用。
使用单个 GPU.


### 载入模型

In [5]:
from models.TimesNet import Model
import torch
import os


def build_model(args):
    # 使用 Args 实例来构建 Model 实例
    model = Model(args)
    return model

def acquire_device(args):
    if args.use_gpu:
        os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu) if not args.use_multi_gpu else args.devices
        device = torch.device(f'cuda:{args.gpu}')
        print(f'Use GPU: cuda:{args.gpu}')
    else:
        device = torch.device('cpu')
        print('Use CPU')
    return device

# 构建模型
device = acquire_device(args)
model = build_model(args).to(device)
model.load_state_dict(torch.load(os.path.join('./checkpoints/' + setting, 'checkpoint.pth'), map_location=torch.device(device)))


Use GPU: cuda:0


<All keys matched successfully>

In [6]:
from sklearn.preprocessing import StandardScaler

all_data_raw = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储原始数据
all_data_scaled = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储标准化后的数据
processed_count = 0  # 初始化计数器
total_count = len(stock_list)  # 获取总股票数量
scaler = StandardScaler()

for i in stock_list[:]:
    NUM = i
    try:
        # 下载原始数据
        raw_data = download_data(NUM, args)
        # 更新已处理股票数量计数器
        processed_count += 1
        # 检查数据长度，如果小于300则跳过此次循环
        if raw_data.shape[0] < 280:
            print(f"股票代码 {NUM} 的数据长度小于300,跳过此次循环。")
            continue
        
        # 拼接数据，添加各种参数
        ad_data = add_data(raw_data, args)
        # 添加预测标签
        # ad_data = add_label(ad_data, args)
        # 删除无效数据
        final_data_raw = sub_data(ad_data.copy(), args)
        
        # 保留最后200行
        final_data_raw = final_data_raw.tail(200)

        file_name_raw_individual = f"raw_stock_cy_{NUM}.csv"
        final_data_raw.to_csv(args.pred_path + file_name_raw_individual, index=False)
        print(f"股票代码 {NUM} 的数据保存到 {file_name_raw_individual} ，形状: {final_data_raw.shape}")
        
        
        # 执行预测
        flag='test'
        data_set = Dataset_Custom(
            root_path=args.pred_path,
            data_path=file_name_raw_individual,
            flag=flag,
            size=[args.seq_len, args.label_len, args.pred_len],
            features=args.features,
            target=args.target,
            scale = args.scale,
            timeenc=1,
            freq=args.freq,
            test_ratio = args.test_ratio,
            seasonal_patterns=args.seasonal_patterns,
        )
        print(flag, len(data_set))

        data_loader = DataLoader(
            data_set,
            batch_size=1,
            shuffle=False,
            num_workers=args.num_workers,
            drop_last=True)
        # 最后一笔数据预测
        preds = []
        trues = []
        model.eval()
        with torch.no_grad():
            data_loader_length = len(data_loader)
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(data_loader):
                if i < data_loader_length - 1 and args.pred_once:
                    continue  # 跳过除了最后一批数据之外的所有批次
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float().to(device)

                batch_x_mark = batch_x_mark.float().to(device)
                batch_y_mark = batch_y_mark.float().to(device)

                # decoder input 
                dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)

                # 打印更多的变量
                # print("dec_inp:", dec_inp)
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                f_dim = -1 if args.features == 'MS' else 0
                outputs = outputs[:, -args.pred_len:, :]
                batch_y = batch_y[:, -args.pred_len:, :].to(device)
                outputs = outputs.detach().cpu().numpy() # 将outputs转换为numpy格式
                batch_y = batch_y.detach().cpu().numpy() # 将batch_y转换为numpy格式
                if data_set.scale and args.inverse: # 如果数据进行了归一化且需要逆归一化
                    shape = outputs.shape
                    outputs = data_set.inverse_transform(outputs.squeeze(0)).reshape(shape)
                    batch_y = data_set.inverse_transform(batch_y.squeeze(0)).reshape(shape)

                outputs = outputs[:, :, f_dim:]
                batch_y = batch_y[:, :, f_dim:]
                
                pred = outputs
                true = batch_y
                
                preds.append(pred)
                trues.append(true)
                
                preds = np.array(preds) # 将预测结果转换为numpy格式
                trues = np.array(trues) # 将真实结果转换为numpy格式
                print('test shape:', preds.shape, trues.shape)
                preds = preds.reshape(-1, preds.shape[-2], preds.shape[-1]) # 将预测结果转换为三维矩阵
                trues = trues.reshape(-1, trues.shape[-2], trues.shape[-1])
                print('test shape:', preds.shape, trues.shape)
        if args.use_gpu:
            torch.cuda.empty_cache()

        # 使用 flatten 方法将 preds 和 trues 转换为一维数组
        preds_flat = preds.flatten()
        trues_flat = trues.flatten()

        # 将一维数组转换为列表
        preds_list = preds_flat.tolist()
        trues_list = trues_flat.tolist()
        
        # 计算从第n天（第1天）到n+5天（第5天）的总变化
        total_preds = calculate_total_change(preds_list, 1, 5)
        total_trues = calculate_total_change(trues_list, 1, 5)
        accuracy = 1 if np.sign(total_preds) == np.sign(total_trues) else 0

        #调整
        accuracy_list = [1 if np.sign(preds_list[i]) == np.sign(trues_list[i]) else 0 for i in range(5)]
        
        # 定义要写入的数据
        data = [NUM]
        for i in range(5):
            data.extend([preds_list[i], trues_list[i], accuracy_list[i]])
        data.extend([total_preds, total_trues, accuracy])

        # 定义列名
        columns = ['股票代码']
        for i in range(1, 6):
            columns.extend(['预测{}天涨幅'.format(i), '真实{}天涨幅'.format(i), '{}天涨幅准确率'.format(i)])

        columns.extend(['预测5天涨跌幅合计', '真实5天涨跌幅合计', '5天涨跌幅准确率'])
        
        # 创建DataFrame
        df = pd.DataFrame([data], columns=columns)
        
        # 检查文件是否存在
        file_path = args.pred_path + 'result.csv'
        file_exists = os.path.isfile(file_path)
        # 如果文件不存在，则写入表头
        if not file_exists:
            df.to_csv(file_path, index=False, encoding='utf-8-sig')
        else:
            df.to_csv(file_path, mode='a', header=False, index=False, encoding='utf-8-sig')

        print(f'股票{NUM}预测结果已保存到 {file_path} 。')
        
        

    except Exception as e:
        print(f"处理股票代码 {NUM} 时出现错误: {e}")
        continue

    # 计算并打印处理进度
    progress = processed_count / total_count
    print(f"处理进度: {processed_count}/{total_count} ({progress:.2%})")



获取数据时间为： 20151201 - 20240105
原始数据形状： (152, 9)
股票代码 301337 的数据长度小于300,跳过此次循环。
获取数据时间为： 20151201 - 20240105
原始数据形状： (95, 9)
股票代码 301511 的数据长度小于300,跳过此次循环。
获取数据时间为： 20151201 - 20240105
原始数据形状： (381, 9)
添加数据以后形状： (381, 21)
删除指定行、列后数据形状:  (278, 17)
股票代码 301238 的数据保存到 raw_stock_cy_301238.csv ，形状: (200, 17)
test 96
test shape: (1, 1, 5, 1) (1, 1, 5, 1)
test shape: (1, 5, 1) (1, 5, 1)
股票301238预测结果已保存到 ./dataset/Stock/pred/result.csv 。
处理进度: 3/591 (0.51%)
获取数据时间为： 20151201 - 20240105
原始数据形状： (664, 9)
添加数据以后形状： (664, 21)
删除指定行、列后数据形状:  (561, 17)
股票代码 300968 的数据保存到 raw_stock_cy_300968.csv ，形状: (200, 17)
test 96
test shape: (1, 1, 5, 1) (1, 1, 5, 1)
test shape: (1, 5, 1) (1, 5, 1)
股票300968预测结果已保存到 ./dataset/Stock/pred/result.csv 。
处理进度: 4/591 (0.68%)
获取数据时间为： 20151201 - 20240105
原始数据形状： (111, 9)
股票代码 301172 的数据长度小于300,跳过此次循环。
获取数据时间为： 20151201 - 20240105
原始数据形状： (8, 9)
股票代码 301526 的数据长度小于300,跳过此次循环。
获取数据时间为： 20151201 - 20240105
原始数据形状： (1854, 9)
添加数据以后形状： (1854, 21)
删除指定行、列后数据形状:  (1751, 17)
股票代码 

KeyboardInterrupt: 